In [1]:
import sqlite3
import pandas as pd
import numpy as np

# Connecting to SQLite database 
conn = sqlite3.connect('/Users/abhinandandas/Downloads/BNZ12.sqlite')

# Data loading into pandas DataFrame
df = pd.read_sql_query("SELECT * FROM tickdata", conn)
conn.close()

# Converting DateTime to required datetime formatby adding milliseconds
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

# New column for date 
df['Date'] = df['DateTime'].dt.date

# Specifying date range for the task
start = pd.to_datetime('2012-09-05').date()
end = pd.to_datetime('2012-12-04').date()

# Filtering based on range given
df = df[(df['Date'] >= start) & (df['Date'] <= end)]

# list for storing results
results = []

# Calculating for each day
for date, group in df.groupby('Date'):
    first_trade = group.iloc[0]
    first_price = first_trade['Price']
    
    # Fetching trades for the required timeframe-- 17:14:00 and 17:15:00
    vwap_trades = group[(group['DateTime'].dt.time >= pd.to_datetime('17:14:00').time()) & 
                        (group['DateTime'].dt.time < pd.to_datetime('17:15:00').time())]
    
    # Calculating Volume-weighted-average price(vwap) using numpy functions
    if not vwap_trades.empty:
        vwap = np.sum(vwap_trades['Price'] * vwap_trades['LotsTraded']) / np.sum(vwap_trades['LotsTraded'])
    else:
        vwap = np.nan
    
    # Checking if first price is traded again after 17:15:00
    posttime_trades = group[group['DateTime'].dt.time >= pd.to_datetime('17:15:00').time()]
    if first_price in posttime_trades['Price'].values:
        firstpost_time = posttime_trades[posttime_trades['Price'] == first_price].iloc[0]['DateTime']
        traded_again = True
        firstpost_time = firstpost_time.time()
    else:
        traded_again = False
        firstpost_time = ''

    # Appending required values for the task
    results.append([date, first_price, vwap, traded_again, firstpost_time])

# New dataframe for extracting results
results_df = pd.DataFrame(results, columns=['Date', 'FirstPrice', 'VWAP_17_14_17_15', 'Traded_Again', 'Firstpost_17:15:00'])

# exporting dataframe to csv
results_df.to_csv('daily_trade_analysis.csv', index=False)